In [5]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
from collections import Counter

words0 = (
    df_train[df_train['target'] == 0]['text'].
    str.lower().
    str.replace('^\w\s',' ').
    str.split()
)
words1 = (
    df_train[df_train['target'] == 1]['text'].
    str.lower().
    str.replace('^\w\s',' ').
    str.split()
)

total0 = list()
for i in words0:
    total0.extend(i)
total1 = list()
for i in words1:
    total1.extend(i)

#total = list()
#for i in words:
    #total.extend(i)

#bag_of_words = words.apply(Counter)
#total[:10]

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


In [7]:
bigrams0 = Counter(get_bigrams(total0))
bigrams1 = Counter(get_bigrams(total1))

test = pd.Series()
test['0'] = total0
test['1'] = total1
test

NameError: name 'get_bigrams' is not defined

In [ ]:
def get_bigrams(words):
    return zip(words, words[1:])

bigrams = test.apply(get_bigrams).apply(Counter)
bigrams

In [ ]:
bag_of_words = test.apply(Counter)
bag_of_words

In [ ]:
tf = pd.DataFrame(list(bag_of_words),index=bag_of_words.index)
tf

In [ ]:
np.sum(np.sum(tf.isnull())) / tf.size

In [ ]:
tf = tf.fillna(0)
tf

In [ ]:
text_series = test.str.join(' ')
text_series

In [8]:
temporary0 = df_train[df_train['target'] == 0]['text'].str.cat(sep='\n')
temporary1 = df_train[df_train['target'] == 1]['text'].str.cat(sep='\n')
temporary_all = pd.Series()
temporary_all['0'] = temporary0
temporary_all['1'] = temporary1
temporary_all

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


0    What's up man?\nI love fruits\nSummer is lovel...
1    Our Deeds are the Reason of this #earthquake M...
dtype: object

### Implementation using ``scikit-learn``

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(temporary_all) # This determines the vocabulary.
tf_sparse = vec.transform(temporary_all)

tf_sparse

<2x21637 sparse matrix of type '<class 'numpy.int64'>'
	with 25776 stored elements in Compressed Sparse Row format>

In [10]:
pd.DataFrame(
    tf_sparse.todense(),
    columns=vec.get_feature_names()
)

,00,000,0000,007npen6lg,00cy9vxeff,00end,00pm,01,02,0215,...,ûò,ûò800000,ûòthe,ûòåêcnbc,ûó,ûóher,ûókody,ûónegligence,ûótech,ûówe
0,8,3,0,0,1,1,0,0,5,1,...,18,0,0,1,16,0,0,0,1,0
1,25,1,1,1,0,0,1,26,2,0,...,24,1,1,0,12,1,1,1,0,1


# Let's try this again, shall we?

In [1]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
corpus = df_train['text'].fillna('')
corpus

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

vec = TfidfVectorizer(norm=None)
vec.fit(corpus)
tf_idf_sparse = pd.DataFrame.sparse.from_spmatrix(vec.transform(corpus))

neigh = KNeighborsClassifier(n_neighbors=11,metric='cosine')
neigh.fit(tf_idf_sparse, df_train['target'])

KNeighborsClassifier(metric='cosine', n_neighbors=11)

In [4]:
def predict_disaster(tweet):
    return neigh.predict(vec.transform([tweet]))[0]

In [15]:
predict_disaster(df_train['text'].loc[0])

1

In [5]:
df_train['pls'] = df_train['text'].apply(predict_disaster)
df_train.head()

,id,keyword,location,text,target,pls
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,1


In [6]:
len(df_train[df_train['target'] == df_train['pls']]) / len(df_train)

0.8188624720872192

In [7]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
submission1 = pd.read_csv('sample_submission.csv')
submission1.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [9]:
submission1['target'] = df_test['text'].apply(predict_disaster)
submission1.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [10]:
submission1.to_csv('submission2.csv', index=False)

### Testing different n values

In [40]:
neigh2 = KNeighborsClassifier(n_neighbors=3,metric='euclidean')
neigh2.fit(tf_idf_sparse, df_train['target'])

KNeighborsClassifier(metric='euclidean', n_neighbors=3)

In [41]:
def predict_disaster(tweet):
    return neigh2.predict(vec.transform([tweet]))[0]

In [42]:
df_train['pls'] = df_train['text'].apply(predict_disaster)
df_train.head()

,id,keyword,location,text,target,pls
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0


In [43]:
len(df_train[df_train['target'] == df_train['pls']]) / len(df_train)

0.7564691974254565

In [39]:
submission2 = pd.read_csv('sample_submission.csv')
submission2['target'] = df_test['text'].apply(predict_disaster)
submission2.to_csv('submission2.csv', index=False)